# Main Code

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from heapq import nlargest
from sklearn.metrics import accuracy_score

from tqdm import tqdm

import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer as optim   
from torch.utils.data import Dataset, DataLoader
from torch.nn import MSELoss

In [2]:
import import_ipynb
from network.FullyConnected_Network import FCnet
from network.Transformer_Network import ATnet
from network.Convolutional_Network import RNnet

importing Jupyter notebook from /mnt/batch/tasks/shared/LS_root/mounts/clusters/rhs/code/Users/qw07020821/NN_DL/nonogram_DL/network/FullyConnected_Network.ipynb
importing Jupyter notebook from /mnt/batch/tasks/shared/LS_root/mounts/clusters/rhs/code/Users/qw07020821/NN_DL/nonogram_DL/network/Transformer_Network.ipynb
importing Jupyter notebook from /mnt/batch/tasks/shared/LS_root/mounts/clusters/rhs/code/Users/qw07020821/NN_DL/nonogram_DL/network/Convolutional_Network.ipynb


## Get Data

In [3]:
print('loading...', flush = True)
X_train = np.load('./data/X_train.npy', allow_pickle= True)  # X_train and X_test is pickled
Y_train = np.load('./data/Y_train.npy')
X_test = np.load('./data/X_test.npy', allow_pickle= True)
Y_test = np.load('./data/Y_test.npy')
print('done')

loading...
done


## Preprocessing - converting conditions into 'possibility maps'

In [4]:
X_train = X_train[0:5000]
Y_train = Y_train[0:5000]
X_test = X_test[0:100]
Y_test = Y_test[0:100]

In [5]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((5000, 2, 10), (5000, 10, 10), (100, 2, 10), (100, 10, 10))

In [6]:
def searchcombinationsUtil(k, n):
    """
    k: number of elements (>= 1)
    n: total sum of elements
    return all possible combinations of k numbers that add up to n, regarding its order
    """
    # Recursive function
    
    if k == 1:
        return [[n]]
    else:
        output = []
        for i in range(0, n+1):
            output += [[i]+items for items in searchcombinationsUtil(k-1, n-i)]
        return output        

In [7]:
def pixel_val_calculator(constraint, N):
    """
    constraint: the condition(constraint) of a single row/column in a nonogram
    N: the length of the width/height of the nonogram
    returns a vector with length N, and each value of the vector is the possibility of the corresponding pixel to be colored
    """
    total_colored = np.sum(constraint)

    if(len(constraint) == 0):
        return [0 for _ in range(N)]
    else:
        combinations = searchcombinationsUtil(k=int(len(constraint)+1), n= int(N-total_colored-len(constraint)+1))
        output = []

        for each_combination in combinations:
            pixel_val = []
            for idx, elements in enumerate(each_combination):
                if (idx == 0) or (idx == len(constraint)):
                    pixel_val += [0 for _ in range(elements)]
                else:
                    pixel_val += [0 for _ in range(elements+1)]
                if idx != (len(constraint)):
                    pixel_val += [1 for _ in range(constraint[idx])]
            output.append(pixel_val)
        
        output = np.array(output, dtype = np.float64)
        output = np.sum(output, axis = 0)
        output/= len(combinations)

        return output

In [8]:
pixel_val_calculator([7, 1], 10)

array([0.66666667, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.33333333, 0.33333333, 0.66666667])

In [9]:
def possibility_map_generator(X):
    """
    X: (batch_size, 2, N) or (batch_size, 2, N, t) only if all constraints are composed of same number of blocks (=t)
    (2, N): each nonogram puzzle
    2 stands for each condition (row condition, column condition)
    N stands for the number of pixels (Number of total constraints)
    returns possibility_map: (batch_size, 2, N, N)
    """
    assert X.ndim in [3, 4]
    
    if X.ndim == 3:
        N = X.shape[-1]
    if X.ndim == 4:
        N = X.shape[-2]

    possibility_map = []
    for puzzles in tqdm(X):
        row_condition = puzzles[0]
        row_map = []
        for constraints in row_condition:
            row_map.append(pixel_val_calculator(constraint=constraints, N=N))
        row_map = np.array(row_map)

        column_condition = puzzles[1]
        column_map = []
        for constraints in column_condition:
            column_map.append(pixel_val_calculator(constraint=constraints, N=N))
        column_map = np.array(column_map)
        column_map = column_map.T

        possibility_map.append(np.array([row_map, column_map]))
        
    possibility_map = np.asarray(possibility_map)
    return possibility_map

In [10]:
a = np.array([[[2], [1]], [[1], [2]]])
print(a.shape)
possibility_map_generator(np.expand_dims(a,0))

100%|██████████| 1/1 [00:00<00:00, 1266.01it/s]


(2, 2, 1)


array([[[[1. , 1. ],
         [0.5, 0.5]],

        [[0.5, 1. ],
         [0.5, 1. ]]]])

## Neural Network

In [11]:
class NN_Dataset(Dataset):

    def __init__(self, X, X_mapped=None, y=None):

        assert X.ndim in [3, 4]

        if X.ndim == 3:
            N = X.shape[-1]
            self.X = np.array([[np.array([k + [0]*(N - len(k))for k in j]) for j in i] for i in X])
        if X.ndim == 4:
            N = X.shape[-2]


        # pre-calculated possibility map if possible
        if X_mapped is not None:
            self.X_mapped = X_mapped
        else:
            self.X_mapped = possibility_map_generator(X)

        self.y = y
        
    def __getitem__(self, idx):
        inputs = np.array(self.X[idx])
        mapped_inputs = np.array(self.X_mapped[idx], dtype = np.float64)
        

        if self.y is not None:    # Train
            labels = self.y[idx]
            labels = np.array(labels, dtype=np.float64)
            return inputs, mapped_inputs, labels

        else:                     # Test
            return inputs, mapped_inputs
    
    def __len__(self):
        return len(self.X)


In [12]:
def get_loader(batch_size, shuffle, num_workers, X, X_mapped=None, y=None):
    dataset = NN_Dataset(X, X_mapped, y)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
                            num_workers=num_workers)

    print(f'length : {len(dataset)}')
    return data_loader

## Training

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [14]:
def train(epochs, model, optimizer, train_loader, valid_loader, device):
    start = time.time()

    best_acc = 0
    for epoch in range(1, epochs+1):

        # Train
        model.train()    
        train_accuracies = []
        train_losses = []
        for i, (inputs, mapped_inputs, labels) in enumerate(tqdm(train_loader)):
            
            optimizer.zero_grad()
            mapped_inputs = torch.tensor(mapped_inputs, device=device, dtype=torch.float32)
            labels = torch.tensor(labels, device=device, dtype=torch.float32)
            
            preds = model(mapped_inputs)

            loss = MSELoss()
            l = loss(preds, labels)
            
            l.backward()
            optimizer.step()

            preds = torch.flatten(preds, start_dim = 1).detach().cpu().numpy()
            labels = torch.flatten(labels, start_dim = 1).detach().cpu().numpy()


            for idx, predictions in enumerate(preds):
                total_colored = np.sum(np.array(inputs[idx][0]))
                threshold = np.amin(nlargest(total_colored, predictions))
                predictions = [1 if a >= threshold else 0 for a in predictions]
                acc = accuracy_score(predictions, labels[idx])
                train_accuracies.append(acc)
            
            
            train_losses.append(l.item())
        

        # Validation
        model.eval()
        val_accuracies = []
        val_losses = []
        with torch.no_grad():
            for inputs, mapped_inputs, labels in valid_loader:
                
                mapped_inputs = torch.tensor(mapped_inputs, device=device, dtype=torch.float32)
                labels = torch.tensor(labels, device=device, dtype=torch.float32)

                
                preds = model(mapped_inputs)


                loss = MSELoss()
                l = loss(preds, labels)
                
                val_losses.append(l.item())

                preds = torch.flatten(preds, start_dim = 1).detach().cpu().numpy()
                labels = torch.flatten(labels, start_dim = 1).detach().cpu().numpy()


                for idx, predictions in enumerate(preds):
                    total_colored = np.sum(np.array(inputs[idx][0]))
                    threshold = np.amin(nlargest(total_colored, predictions))
                    predictions = [1 if a >= threshold else 0 for a in predictions]
                    acc = accuracy_score(predictions, labels[idx])
                    val_accuracies.append(acc)
            
            val_losses.append(l.item())

            
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        train_acc = np.mean(train_accuracies)
        val_acc = np.mean(val_accuracies)

        if best_acc < val_acc:
            best_acc = val_acc
            best_epoch = epoch
            print('saving model...', flush = True)
            if isinstance(model, FCnet):
                torch.save(model.state_dict(), './weights/FCnet.pth')
            if isinstance(model, RNnet):
                torch.save(model.state_dict(), './weights/RNnet.pth')
            print('done')

        print(f'Epoch:{epoch}  Train Loss:{train_loss:.3f} | Valid Loss:{val_loss:.3f}')
        print(f'Train  Acc:{train_acc:.3f}')
        print(f'Valid  Acc:{val_acc:.3f}')

    end = time.time()
    print(f'\nEpoch Process Time: {(end-start)/60:.2f}Minute')
    print(f'Best Epoch:{best_epoch}, Best Acc:{best_acc:.3f}')

In [15]:
X_train_mapped = possibility_map_generator(X_train)
X_test_mapped = possibility_map_generator(X_test)

X_train, X_val, X_train_mapped, X_val_mapped, Y_train, Y_val = train_test_split(X_train, X_train_mapped, Y_train, test_size = 0.2, random_state = 42) 

100%|██████████| 100/100 [00:00<00:00, 298.20it/s]


In [16]:
print(X_train.shape, X_val.shape, X_train_mapped.shape, X_val_mapped.shape, Y_train.shape, Y_val.shape)

(4000, 2, 10) (1000, 2, 10) (4000, 2, 10, 10) (1000, 2, 10, 10) (4000, 10, 10) (1000, 10, 10)


In [17]:
train_loader = get_loader(batch_size=128, shuffle=True, num_workers=6, X = X_train, X_mapped=X_train_mapped, y = Y_train) 
valid_loader = get_loader(batch_size=128, shuffle=False, num_workers=6, X = X_val, X_mapped = X_val_mapped, y = Y_val) 
epochs = 100
lr = 0.0003

num_pixels = X_train.shape[-2] if X_train.ndim is 4 else X_train.shape[-1]

#model = MLP_Network(num_pixels = num_pixels).to(device)
# model = RNnet(n_channels = 2).to(device)

ATnet_Config = {
    'N': 10,
    'emb_dim': 64,
    'depth': 4, 
    'd_model': 64, 
    'num_heads': 8, 
    'expansion': 4, 
    'dropout': 0.1
}

model = ATnet(**ATnet_Config).to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr)

length : 4000
length : 1000


In [18]:
best_epoch, best_auprc, best_auc = train(epochs = epochs, model = model, optimizer = optimizer, train_loader = train_loader, \
                                         valid_loader = valid_loader, device = device)

  0%|          | 0/32 [00:00<?, ?it/s]/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
100%|██████████| 32/32 [00:02<00:00, 12.17it/s]
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/anaconda/envs/azureml_py36/lib/pyth

saving model...
done
Epoch:1  Train Loss:0.169 | Valid Loss:0.157
Train  Acc:0.702
Valid  Acc:0.723
saving model...
done
Epoch:2  Train Loss:0.152 | Valid Loss:0.147
Train  Acc:0.741
Valid  Acc:0.767
saving model...
done
Epoch:3  Train Loss:0.141 | Valid Loss:0.132
Train  Acc:0.775
Valid  Acc:0.792
saving model...
done
Epoch:4  Train Loss:0.132 | Valid Loss:0.127
Train  Acc:0.792
Valid  Acc:0.801
Epoch:5  Train Loss:0.129 | Valid Loss:0.127
Train  Acc:0.796
Valid  Acc:0.801
saving model...
done
Epoch:6  Train Loss:0.129 | Valid Loss:0.126
Train  Acc:0.796
Valid  Acc:0.802
Epoch:7  Train Loss:0.128 | Valid Loss:0.126
Train  Acc:0.797
Valid  Acc:0.802
Epoch:8  Train Loss:0.128 | Valid Loss:0.125
Train  Acc:0.798
Valid  Acc:0.802
saving model...
done
Epoch:9  Train Loss:0.127 | Valid Loss:0.125
Train  Acc:0.798
Valid  Acc:0.802
saving model...
done
Epoch:10  Train Loss:0.127 | Valid Loss:0.125
Train  Acc:0.799
Valid  Acc:0.803
Epoch:11  Train Loss:0.126 | Valid Loss:0.125
Train  Acc:0.800

KeyboardInterrupt: 

## Inference

In [ ]:
def inference(model, test_loader, device):
    pred_list = []

    model.eval()
    with torch.no_grad():
        for inputs, mapped_inputs in test_loader:
            
            mapped_inputs = torch.tensor(mapped_inputs, device=device, dtype=torch.float32)
            
            N = mapped_inputs.shape[-1]
            preds = model(mapped_inputs)
            preds = preds.detach().cpu().numpy()

            
            for idx, predictions in enumerate(preds):
                total_colored = np.sum(np.array(inputs[idx][0]))
                threshold = np.amin(nlargest(total_colored, predictions))
                predictions = [1 if a >= threshold else 0 for a in predictions]
                pred_list.append(np.array(predictions).reshape((N, N)))

        pred_list = np.array(pred_list)
        
    return pred_list

In [ ]:
num_pixels = X_test.shape[-2] if X_test.ndim is 4 else X_test.shape[-1]

model = MLP_Network(num_pixels = num_pixels)
model.load_state_dict(torch.load('./weights/MLP.pth'))
model.to(device)

test_loader = get_loader(batch_size=32, shuffle=False, num_workers=6, X = X_test, X_mapped = X_test_mapped, y = None) 

pred_list = inference(model = model, test_loader = test_loader, device = device)



In [50]:
i = 10

print(pred_list[i])
print(Y_test[i])

[[0 1 1 0 0 1 1 1]
 [0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1]
 [1 1 0 1 0 0 1 0]
 [1 1 0 1 1 0 1 1]]
[[0 0 1 1 0 0 1 1]
 [0 1 0 0 0 0 1 1]
 [0 0 0 0 0 1 0 1]
 [1 0 0 1 0 0 0 1]
 [0 0 1 0 0 1 0 0]
 [0 1 0 0 0 0 1 0]
 [1 0 0 1 0 0 1 0]
 [1 1 0 1 1 0 1 1]]
